In [1]:
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
class Model:
    def __init__(self, layers=None):
        if not layers:
            layers = list()
            
        for layer in layers:
            if not isinstance(layer, Layer):
                raise TypeError('Argument is not a layer.')
        
        self.__layers = layers
        self.__fitted = False
        
    def add(self, layer):
        if self.__fitted:
            raise AttributeError('The model has already been fitted.')
        
        if not isinstance(layer, Layer):
            raise TypeError('Argument is not a layer.')
            
        self.__layers.append(layer)
        
        return self
        
    def fit(self, X, Y, epochs=10, eta=.01):
        X = np.array(X); Y = np.array(Y)
        self.__input_shape = X[0].shape
        
        
        
        if X.shape[0] != Y.shape[0]:
            raise ValueError('Inputs have different shape.')
        
        for layer in self.__layers:
            layer._set_input_shape(self.__input_shape)
            
                
        for epoch in range(epochs):   
            for i in range(len(X)):
                x = X[i]; y = Y[i];
                for layer in self.__layers:
                    x = layer._net_input(x)
                        
                update = eta * (y - x)   
                    
                for layer in self.__layers:
                    layer._update(update)  
                    
        self.__fitted = True            
        return self            
            
    def predict(self, X):
        if not self.__fitted: raise TypeError('The model is not fitted.') 
        return [1 if self._net_input(x) > 0 else 0 for x in X]
    
    def _net_input(self, x):
        for layer in self.__layers:
            x = layer._net_input(x)
            
        return x.sum() 
    
    def score(self, X, Y):
        return sum(np.array(self.predict(X)) == Y) / len(Y) 

In [3]:
class Layer:
        
    def _net_input(self, x):
        if not self.__input_shape == x.shape:
            raise ValueError(f'X has bad shape {x.shape}, must be {self.__input_shape}')
            
        self.__input = x    
            
        return x * self.__weights + self.__intercept   
    
    def _set_input_shape(self, shape):
        self.__input_shape = shape
        self.__weights = np.random.random(self.__input_shape)
        self.__intercept = 0
        
    def _update(self, update):
        delta_w = update * self.__input
        
        self.__weights += delta_w
        self.__intercept += delta_w
        

In [4]:
data = load_breast_cancer()

X = data.data
y = data.target

In [5]:
model = Model()
model.add(Layer())
model.add(Layer())
model.add(Layer())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

model.fit(X_train, y_train, epochs=500, eta=0.1)

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in multiply
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in multiply
  if __name__ == '__main__':
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in add
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in add
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in add
  if __name__ == '__main__':


In [6]:
model.score(X_test, y_test)

0.49122807017543857